# HW3 Image Classification
## We strongly recommend that you run with Kaggle for this homework
https://www.kaggle.com/c/ml2022spring-hw3b/code?competitionId=34954&sortBy=dateCreated

# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [1]:
# ! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip

In [2]:
# ! unzip food11.zip

# Training

In [1]:
_exp_name = "sample_batch64_withlr"

In [2]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.autograd import Variable
import torchvision.models as models
from sklearn.model_selection import KFold
# import optuna
# from optuna.trial import TrialState

from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
import random

/home/master/10/tracyhuang/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
myseed = 58961  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(myseed)
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [4]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = A.Compose([
    A.Resize(256,256),
    A.CenterCrop(224,224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = A.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    A.Resize(256,256),
    A.CenterCrop(224,224),
    A.CLAHE(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=0.5),
    A.Blur(blur_limit=3,p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])


In [7]:
# ref:https://github.com/facebookresearch/mixup-cifar10/blob/main/train.py
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [5]:
class FoodDataset(Dataset):

    def __init__(self,path,train_tfm=train_tfm,test_tfm=test_tfm,files = None, mode='train'):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.train_transform = train_tfm
        self.test_transform = test_tfm
        self.mode = mode
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        # im = Image.open(fname)
        # im = self.transform(im)
        im = cv2.imread(fname)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

        
        train_augmented = self.train_transform(image=im)
        train_im = train_augmented['image']
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        
        if self.mode=='train':
            return train_im,label
        else:
            test_augmented = self.test_transform(image=im)
            test_im = test_augmented['image']
            train_ims = [train_im]
            for _ in range(4):
                temp = self.train_transform(image=im)['image']
                train_ims.append(temp)
            return train_ims, test_im, label



In [6]:
BATCHSIZE = 64
_dataset_dir = "./food11"

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.

# train_set = FoodDataset(os.path.join(_dataset_dir,"training"), mode='train')
# train_loader = DataLoader(train_set, batch_size=BATCHSIZE, shuffle=True, num_workers=0, pin_memory=True)
# valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), mode='test')
# valid_loader = DataLoader(valid_set, batch_size=BATCHSIZE, shuffle=True, num_workers=0, pin_memory=True)
train_set = FoodDataset(os.path.join(_dataset_dir,"training"), mode='train')
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), mode='train')
concat_data = ConcatDataset([train_set, valid_set])

One ./food11/training sample ./food11/training/0_0.jpg
One ./food11/validation sample ./food11/validation/0_0.jpg


In [7]:
def tta(model, train_ims, test_im):
    test_tfm_pred = model(test_im.to(device))
    train_tfm_preds =[]
    for im in train_ims:
        temp = model(im.to(device))
        train_tfm_preds.append(temp)
    
    avg_train = sum(train_tfm_preds) / len(train_tfm_preds)
    return 0.5 * avg_train + 0.5 * test_tfm_pred

In [11]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')

In [12]:
# "cuda" only when GPUs are available.
torch.cuda.set_device(1)
device = "cuda" if torch.cuda.is_available() else "cpu"
k_folds = 5
# The number of training epochs and patience.
n_epochs = 170

 # Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)
# Initialize a model, and put it on the device specified.
# model = Classifier().to(device)
for fold, (train_ids, test_ids) in enumerate(kfold.split(concat_data)):


    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    # valid_loader = DataLoader(valid_set, batch_size=BATCHSIZE, shuffle=True, num_workers=0, pin_memory=True)
    train_loader = DataLoader(
                      concat_data, 
                      batch_size=BATCHSIZE, sampler=train_subsampler, pin_memory=True, num_workers=2)
    valid_loader = DataLoader(
                      concat_data,
                      batch_size=BATCHSIZE, sampler=valid_subsampler, pin_memory=True, num_workers=2)

    model = models.resnet18(num_classes=11,pretrained=False).to(device)
    model.apply(reset_weights)

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    # For the classification task, we use cross-entropy as the measurement of performance.
    criterion = nn.CrossEntropyLoss()

    # Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=20) 

    # Initialize trackers, these are not parameters and should not be changed
    best_acc = 0

    for epoch in range(n_epochs):

        # ---------- Training ----------
        # Make sure the model is in train mode before training.
        model.train()

        # These are used to record information in training.
        # train_loss = []
        train_accs = []

        for batch in train_loader:

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            imgs, labels = imgs.cuda(), labels.cuda()
            #imgs = imgs.half()
            #print(imgs.shape,labels.shape)
            inputs, targets_a, targets_b, lam = mixup_data(imgs, labels,
                                                        0.5, True)

            # Forward the data. (Make sure data and model are on the same device.)
            inputs, targets_a, targets_b = map(Variable, (inputs,
                                                        targets_a, targets_b))
            logits = model(inputs)

            # Calculate the cross-entropy loss.
            # We don't need to apply softmax before computing cross-entropy as it is done automatically.
            # loss = criterion(logits, labels.to(device))
            loss = mixup_criterion(criterion, logits, targets_a, targets_b, lam)

            # Gradients stored in the parameters in the previous step should be cleared out first.
            optimizer.zero_grad()

            # Compute the gradients for parameters.
            loss.backward()

            # Clip the gradient norms for stable training.
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

            # Update the parameters with computed gradients.
            optimizer.step()
            

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            # train_loss.append(loss.item())
            train_accs.append(acc)
        scheduler.step()
        # train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        # Print the information.
        print(f"Fold:{fold}[ Train | {epoch + 1:03d}/{n_epochs:03d} ] acc = {train_acc:.5f}")

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()

        # These are used to record information in validation.
        # valid_loss = []
        valid_accs = []

        # Iterate the validation set by batches.
        for batch in valid_loader:

            # A batch consists of image data and corresponding labels.
            # train_im, test_im, label
            test_im, labels = batch
            #imgs = imgs.half()

            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model(test_im.to(device))

            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            # valid_loss.append(loss.item())
            valid_accs.append(acc)
            #break

        # The average loss and accuracy for entire validation set is the average of the recorded values.
        # valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"Fold:{fold}[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] acc = {valid_acc:.5f}")

        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"./models/{_exp_name}_foldbest-{fold}.ckpt") # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0

Reset trainable parameters of layer = Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Reset trainable parameters of layer = BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Reset trainable parameters of layer = BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Reset trainable parameters of layer = BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Reset trainable parameters of layer = BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Conv2d(64, 128, kernel_siz

In [8]:
import gc
del train_loader, valid_loader
gc.collect()

NameError: name 'train_loader' is not defined

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), mode='test')
test_loader = DataLoader(test_set, batch_size=BATCHSIZE, shuffle=False, num_workers=0, pin_memory=True)

One ./food11/test sample ./food11/test/0001.jpg


In [88]:
c= torch.tensor(np.array([[1,1],[1,1]]))
d= torch.tensor(np.array([[3,3],[3,3],[3,3]]))
e = [c,d]
o = torch.cat(e)

c2= torch.tensor(np.array([[2,2],[2,2]]))
d2= torch.tensor(np.array([[5,5],[5,5],[5,5]]))
e2 = [c2,d2]
o2 = torch.cat(e2)
print(o2)
print(o)

print(torch.stack([o,o2], dim=0).sum(dim=0))



# a=[]
# for e in c:
#     a+=e
# print(np.array(a).squeeze())

tensor([[2, 2],
        [2, 2],
        [5, 5],
        [5, 5],
        [5, 5]])
tensor([[1, 1],
        [1, 1],
        [3, 3],
        [3, 3],
        [3, 3]])
tensor([[3, 3],
        [3, 3],
        [8, 8],
        [8, 8],
        [8, 8]])


# Testing and generate prediction CSV

In [89]:
results = []
for fold in range(5):
    preds = []
    model_best = models.resnet18(num_classes=11,pretrained=False).to(device)
    model_best.load_state_dict(torch.load(f"./models/{_exp_name}_foldbest-{fold}.ckpt"))
    model_best.eval()
    with torch.no_grad():
        for train_im, test_im, _ in test_loader:
            test_pred = tta(model_best, train_im, test_im)
            preds.append(test_pred)
    results.append(torch.cat(preds))

mixpred = torch.stack(results, dim=0).sum(dim=0) / 5
print(mixpred)
# prediction = []
# model_best = models.resnet18(num_classes=11,pretrained=False).to(device)
# model_best.load_state_dict(torch.load(f"./models/{_exp_name}_foldbest-{fold}.ckpt"))
# model_best.eval()
# with torch.no_grad():
#     for train_im, test_im, _ in test_loader:
#         test_pred = tta(model_best, train_im, test_im)
#         test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
#         prediction += test_label.squeeze().tolist()


tensor([[-0.9823, -2.1671,  1.9977,  ..., -1.8952,  2.0307, -1.3499],
        [-1.1333, -1.6866,  0.1925,  ..., -1.3104,  4.2581, -1.6291],
        [ 4.3901, -1.8886, -0.6039,  ..., -0.9742, -1.3048, -1.7977],
        ...,
        [ 3.7385, -2.0815,  1.0600,  ..., -1.8550,  0.7622, -2.3669],
        [-1.6849, -0.3526,  3.2436,  ..., -0.4111,  0.9675, -2.8746],
        [ 0.3616,  0.5289,  0.2043,  ..., -1.9994,  0.2625, -2.3404]],
       device='cuda:0')


In [92]:
prediction = []
test_label = np.argmax(mixpred.cpu().data.numpy(), axis=1)
prediction += test_label.squeeze().tolist()

In [93]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission_kfold.csv",index = False)

# Q1. Augmentation Implementation
## Implement augmentation by finishing train_tfm in the code with image size of your choice. 
## Directly copy the following block and paste it on GradeScope after you finish the code
### Your train_tfm must be capable of producing 5+ different results when given an identical image multiple times.
### Your  train_tfm in the report can be different from train_tfm in your training code.


In [17]:
# train_tfm = transforms.Compose([
#     # Resize the image into a fixed shape (height = width = 128)
#     transforms.Resize((128, 128)),
#     transforms.RandomCrop((128, 128)),
#     transforms.RandomAffine(degrees=(-30,30), translate=(0, 0.5), scale=(0.4, 0.5), shear=(0,0), fillcolor=(0,255,255)),
#     transforms.ColorJitter(brightness=(0, 5), contrast=(
#         0, 5), saturation=(0, 5), hue=(-0.1, 0.1)),
#     # You need to add some transforms here.
#     transforms.ToTensor(),
# ])

# Q2. Residual Implementation
![](https://i.imgur.com/GYsq1Ap.png)
## Directly copy the following block and paste it on GradeScope after you finish the code


In [18]:
# #ref:https://github.com/FrancescoSaverioZuppichini/ResNet
# from torch import nn
# class Residual_Network(nn.Module):
#     def __init__(self):
#         super(Residual_Network, self).__init__()
        
#         self.cnn_layer1 = nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1, 1),
#             nn.BatchNorm2d(64),
#         )

#         self.cnn_layer2 = nn.Sequential(
#             nn.Conv2d(64, 64, 3, 1, 1),
#             nn.BatchNorm2d(64),
#         )

#         self.cnn_layer3 = nn.Sequential(
#             nn.Conv2d(64, 128, 3, 2, 1),
#             nn.BatchNorm2d(128),
#         )

#         self.cnn_layer4 = nn.Sequential(
#             nn.Conv2d(128, 128, 3, 1, 1),
#             nn.BatchNorm2d(128),
#         )
#         self.cnn_layer5 = nn.Sequential(
#             nn.Conv2d(128, 256, 3, 2, 1),
#             nn.BatchNorm2d(256),
#         )
#         self.cnn_layer6 = nn.Sequential(
#             nn.Conv2d(256, 256, 3, 1, 1),
#             nn.BatchNorm2d(256),
#         )
#         self.fc_layer = nn.Sequential(
#             nn.Linear(256* 32* 32, 256),
#             nn.ReLU(),
#             nn.Linear(256, 11)
#         )
#         self.shortcut = nn.Identity()   
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         # input (x): [BATCHSIZE, 3, 128, 128]
#         # output: [BATCHSIZE, 11]

#         # Extract features by convolutional layers.
#         x1 = self.cnn_layer1(x)
        
#         x1 = self.relu(x1)

#         residual = self.shortcut(x1)
        
#         x2 = self.cnn_layer2(x1)

#         x2 += residual
        
#         x2 = self.relu(x2)
        
#         x3 = self.cnn_layer3(x2)
        
#         x3 = self.relu(x3)

#         residual = self.shortcut(x3)
        
#         x4 = self.cnn_layer4(x3)

#         x4 += residual
        
#         x4 = self.relu(x4)
        
#         x5 = self.cnn_layer5(x4)
        
#         x5 = self.relu(x5)

#         residual = self.shortcut(x5)
        
#         x6 = self.cnn_layer6(x5)

#         x6 += residual
        
#         x6 = self.relu(x6)
        
#         # The extracted feature map must be flatten before going to fully-connected layers.
#         xout = x6.flatten(1)

#         # The features are transformed by fully-connected layers to obtain the final logits.
#         xout = self.fc_layer(xout)
#         return xout